In [ ]:
!pip install instructor

In [ ]:
!pip install evaluate

In [3]:
import os
import instructor
from pydantic import BaseModel
from openai import OpenAI
import getpass

In [ ]:
#API_KEY = getpass.getpass("Enter your OpenAI API key: ")
API_KEY  = # Insert OpenAI Key here
open_ai = OpenAI(api_key=API_KEY)
client  = instructor.from_openai(open_ai, mode=instructor.Mode.TOOLS_STRICT)

In [41]:
class ZeroShotResponse(BaseModel):
    filler: str
    message: str
def ZeroShotPrompting(message):
    return client.chat.completions.create(
        model="gpt-4o-mini",
        response_model=ZeroShotResponse,
        messages=[
            {
                "role": "system",
                "content": """You have excellent knowledge of interactive theorem prover Isabelle .
                              You will try your best to solve the query given by the user.""",
            },
            {
                "role": "user",
                "content": f"""You are given a mathematical statement in natural lanaguage written in LaTex, i.e Natural Language:"{message}". Translate the natural language version to an Isabelle version. You only need to translate the statement into Isabelle not solve it.
                Remember the output that you give is structured as filler and message. The message part should strictly contain only Isabelle translation and nothing else. Everthing else should be in filler. Transalation should only be in Isabelle version.
                """,
            },
        ],
    )

In [6]:
import json

In [7]:
with open("valid_data.json","r") as f:
  data=json.load(f)


In [42]:
class Auto():
  def __init__(self, prompting):
    self.prompting = prompting

  def response(self,dataset):
    y_pred=[]
    y_test=[]
    for message in dataset:
      print(message)
      res=self.prompting(dataset[message]["natural_s"])
      y_pred.append(res.message)
      y_test.append(dataset[message]["formal_s"])
    return y_pred,y_test

  def evaluate(self,y_pred,y_test):
    import evaluate
    metric = evaluate.load("bleu")
    return metric.compute(predictions=y_pred, references=y_test)



In [43]:
agent=Auto(ZeroShotPrompting)

In [ ]:
pred,test=agent.response(data)

In [45]:
agent.evaluate(pred,test)

{'bleu': 0.293757982810891,
 'precisions': [0.5538213526782443,
  0.34268447362530774,
  0.24120373888593358,
  0.1843592721641502],
 'brevity_penalty': 0.9691955049289096,
 'length_ratio': 0.9696603666335086,
 'translation_length': 13647,
 'reference_length': 14074}

In [46]:
count=20

for i in range(len(pred)):
  print("Truth")
  print(test[i])
  print("pred")
  print(pred[i])
  count-=0
  if count==0:
    break


Truth
theorem:
  shows "ceiling (sqrt 27) - floor (sqrt 26) = 1"
  
pred
import Data.Real.

lemma evaluate_sqrt : \<lceil\<sqrt>27\<rceil> - \<lfloor\<sqrt>26\<rfloor> = 1 \<rbrakk>
Truth
theorem:
  fixes a :: "nat \<Rightarrow> rat"
  assumes "a 1 = 1"
    and "a 2 = 3 / 7"
    and "\<forall> n. a (n + 2) = (a n * a (n + 1)) / (2 * a n - a (n + 1))" 
  shows "fst (quotient_of (a 2019)) + snd (quotient_of (a 2019)) = 8078"
  
pred
theory Recursive_Sequence
  imports Main
begin

(* Define the recursive sequence *)
fun a :: "nat \<Rightarrow> rational" where
  "a 1 = 1"
| "a 2 = 3 / 7"
| "a n = (a (n - 2) * a (n - 1)) / (2 * a (n - 2) - a (n - 1)) if n \ge> 3" 


(* Statement to prove *)
value_2019_thm:
  shows "a 2019 = p / q" where "gcd p q = 1" and "p > 0" and "q > 0"
  by (auto)

end
Truth
theorem:
  fixes x :: real
  assumes h0 : "x^3 - (x+1) * (x-1) * x = 5"
  shows "x^3 = 125"

pred
locale cube_volume =
  fixes side_length :: nat
  assumes "side_length > 0"
  defines "volume_cube 

In [ ]:
class AutoCL():
